In [ ]:
import sys
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
sys.path.insert(0,'../scripts/')
from postprocessing import setup_postproc, check_pdc, plot_phi, get_obs_and_noise, get_pars, plot_group, plot_pars_group

In [ ]:
# curr_model = '01473000'
# curr_model = '05431486'
curr_model = '09112500'
# curr_run_root = 'ies'
curr_run_root = 'prior_mc_reweight'
eval_pdc = True
unzip_dirs = False

pstdir, results_file, tmp_res_path, fig_dir, obs, pst = setup_postproc(curr_model, curr_run_root, unzip_dirs)

### check out PDC

In [ ]:
pst.nnz_obs_groups

In [ ]:
if eval_pdc:
    pdc = check_pdc(tmp_res_path, curr_run_root, pst, obs)
    print(pdc)

### look at PHI history

In [ ]:
phi = plot_phi(tmp_res_path, curr_run_root, curr_model, fig_dir)

### Truncate PHI at a threshold

In [ ]:
phi

In [ ]:
best_iter = 0

## now rejection sampling for outlier PHI values

In [ ]:
orgphi = phi.loc[best_iter].iloc[5:].copy()
ax = orgphi.hist(bins=50)
lims = ax.get_xlim()

In [ ]:
# phi_too_high = 2.2e9
# phi_too_high = 0.6e10
# phi_too_high = 1.08e9
# phi_too_high = 7.5e8
phi_too_high = 3.1e9
# phi_too_high = 8.675e8

In [ ]:
phi = orgphi.loc[orgphi<=phi_too_high]
fig,ax = plt.subplots(1,2)
### --> need to indicate which reals we will carry forward <-- ###
orgphi.hist(bins=50, ax=ax[0])
reals = phi.index 
phi.hist(bins=50, ax=ax[1])
ax[0].axvline(phi_too_high, color='orange')
ax[1].set_xlim(lims)
ax[0].set_title(f'Original PHI: {len(orgphi)} reals')
ax[1].set_title(f'Truncated PHI: {len(phi)} reals')
plt.savefig(fig_dir/ 'phi_histogram.pdf')


# How about parameters?

In [ ]:
parens = get_pars(tmp_res_path, curr_run_root, reals, best_iter, pst)


In [ ]:
pargps = parens['pargroup'].unique()
pargps


In [ ]:
for cg in pargps:
    plot_pars_group(parens, cg, fig_dir, curr_model, best_iter, curr_run_root)

### TODO: need to handle the cases without monthly data

# Now let's start looking at the fits

In [ ]:
modens, obens_noise = get_obs_and_noise(tmp_res_path, curr_run_root, reals, best_iter)

In [ ]:
modens

In [ ]:
obens_noise

In [ ]:
pst.obs_groups

In [ ]:
# plot_group('sca_daily', obs, modens, obens_noise, fig_dir, curr_model)

In [ ]:
plot_group('actet_mean_mon', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)
plot_group('actet_mon', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)
plot_group('runoff_mon', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)
plot_group('soil_moist_mon', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)
plot_group('recharge_ann', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)

# streamflow_daily is a special case - all aggregated
plot_group('streamflow_daily', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)
plot_group('streamflow_mean_mon', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)
plot_group('streamflow_mon', obs, modens, obens_noise, fig_dir, curr_model, best_iter, curr_run_root)
